In [62]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import textwrap

from generate import *
from helpers import *

### Import model

In [80]:
model = torch.load('models/simple.model_E400')

### Generate text

In [81]:
text, hiddens = generate(model, '\n\n', 500, 0.8, True, output_hiddens=True)
print(text)

RIVERTH:
The hath and said, your dreay, your and ake.
I tater swords tht I unce sees on the strading:
I Lene difibe: quad mornish him my lall to cannisy;
And the before to my tonch'd no.

RICHARD:
Hoor but thing mastends.

QUEEN MARGARET:
I beat the she may my dear Camongy.

Shenm's and imes:
Who broughts to plaaring?

FORTHS MOFLAND:
Till queenss, then not like what life you slame 'tay be
Unlement low your more bithel ables one thild enjucy,
Oxe: but upul nore carly did Grous.

LORD SANTES:
In 


### Generate big plot with all neurons

In [82]:
def big_plot(text, colors, W, zoom=1.0, cols=4, save_file=None):
    ''' Plot wrapped rnn output with each color representing the activation 
        of a certain node in the hidden representation '''
    N = len(colors)
    rows = math.ceil(N/cols)
    
    plt.figure(figsize=(cols*zoom*W/5, rows*zoom*len(text)/W/3))
    plt.subplots_adjust(wspace = 0.01, hspace = .01)
    for i in range(N):
        plt.subplot(rows, cols, i+1)
        heat = colors[i].flatten()
        wrapped_text, wrapped_heat = wrap_colored_text(text, heat, W=60)
        sns.heatmap(wrapped_heat, annot=wrapped_text, fmt='s',
                    cbar=False, cmap='coolwarm')
        plt.xticks([])
        plt.yticks([])
        plt.title('Node {}'.format(i))
    if save_file:
        plt.savefig(save_file)
    plt.show()

In [83]:
W = 60
big_plot(text, hiddens.T[:20], W=W, zoom=1.0, save_file='plots/neurons-0-20.png')
big_plot(text, hiddens.T[20:40], W=W, zoom=1.0, save_file='plots/neurons-20-40.png')
big_plot(text, hiddens.T[40:60], W=W, zoom=1.0, save_file='plots/neurons-40-60.png')
big_plot(text, hiddens.T[60:80], W=W, zoom=1.0, save_file='plots/neurons-60-80.png')
big_plot(text, hiddens.T[80:100], W=W, zoom=1.0, save_file='plots/neurons-80-100.png')

IndexError: index 20 is out of bounds for axis 0 with size 20